In [348]:
import numpy as np
from scipy.optimize import minimize
from qiskit import *
from qiskit.tools.visualization import plot_histogram
%matplotlib inline

In [349]:
'''alpha0 = 2*np.pi*np.random.random_sample()
theta0 = 2*np.pi*np.random.random_sample()
phi0 = 2*np.pi*np.random.random_sample()'''
alpha0 = 0
theta0 = 0
phi0 = 0

x0 = np.asarray((alpha0, theta0, phi0))
shots = 1000

Any one qubit rotation can be simulated by 5 subsequent rotations around any 2 axes as in definition of loss_function.

In [350]:
def loss_function(x, *args):
    alpha, theta, phi = x
    nshots = args[0]
    
    qr = QuantumRegister(2)
    cr = ClassicalRegister(2)

    circuit = QuantumCircuit(qr,cr)
    
    circuit.rx(-theta,0)
    circuit.ry(-phi,0)
    circuit.rx(alpha,0)
    circuit.ry(phi,0)
    circuit.rx(theta,0)
    
    circuit.cx(qr[0], qr[1])
    circuit.measure(qr,cr)

    simulator = Aer.get_backend('qasm_simulator')
    result = execute(circuit, backend = simulator, shots = nshots).result()

    counts = result.get_counts(circuit)
    l1 = counts.get('00', 0)
    l2 = counts.get('11', 0)
    loss = ((l1-l2)/nshots)**2
    return loss

In [351]:
res1 = minimize(loss_function, x0, args=(shots), method='Powell')

In [352]:
res1

   direc: array([[ 0.        ,  0.        ,  1.        ],
       [ 0.        ,  1.        ,  0.        ],
       [-0.00985943, -0.0109885 , -0.00387438]])
     fun: array(1.6e-05)
 message: 'Optimization terminated successfully.'
    nfev: 101
     nit: 2
  status: 0
 success: True
       x: array([ 1.54192963,  1.17292   , -2.58506334])

In [353]:
loss_function(res1['x'],shots)

0.004096

The loss function I chose is the difference between numbers of shots resulting in 00 and 11 per total number of shots in the run of the experiment squared. I found that the loss decreased with the number of shots in the experiment (as expected).

We can check whether the resulting state is 00 + 11 by applying a Hadamard gate to it (or, better said, our simulation of Hadamard gate, so rotation 2 alpha instead of 1 alpha) and seeing if the result is 00. The loss function could then be sum of our loss function and the difference of the double application (2 alpha rotation) from 00 (difference of the number of shots squared).